In [1]:
# conda install PyAudio
# sudo apt install ffmpeg
# conda install -c conda-forge liblsl
# sudo apt install pulseaudio
# sudo apt install libpcap-dev
# sudo setcap 'cap_net_raw,cap_net_admin=eip' $(which hcitool)
# conda install alsa-plugins
# sudo apt install python3-dev

import subprocess
import threading
import pyaudio
import wave
from pynput import keyboard
from pylsl import StreamInlet, resolve_byprop, resolve_stream
import numpy as np
import os
from datetime import datetime
import csv
import time
from pydub import AudioSegment
import io
import shutil
import boto3

# Global flag to control recording state
recording_active = False

def start_muse_lsl_stream():
    """Start the muse2 stream"""
    subprocess.Popen(["muselsl", "stream"], shell=False)

def wait_for_eeg_source(timeout=10):
    """Attempt to find an EEG stream for a given timeout (in seconds)."""
    print("Searching for EEG source...")
    end_time = time.time() + timeout
    while time.time() < end_time:
        try:
            streams = resolve_stream('type', 'EEG')
            if streams:
                print("EEG source found.")
                inlet = StreamInlet(streams[0], max_chunklen=12)
                return inlet
        except Exception as e:
            print(f"Error finding stream: {e}")
        time.sleep(1)
    raise RuntimeError("Failed to find EEG stream within the timeout period.")

def toggle_recording(key):
    """ Toggle recording_active global variable on spacebar press"""
    global recording_active
    if key == keyboard.Key.space: # Check if spacebar is pressed
        if recording_active:
            print("Stopping recordings...")
            recording_active = False
        else:
            print("Starting recordings...")
            recording_active = True

def record_audio(filename):
    """Record audio from the inbuilt microphone and save it as an MP3 file.
    The recording is synchronized with the EEG stream."""

    global recording_active
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    CHUNK = 1024

    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    
    frames = []
    while recording_active:
        data = stream.read(CHUNK, exception_on_overflow=False)
        frames.append(data)

    # Stop recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Convert the recorded data to an audio segment
    recorded_audio = AudioSegment(
        data=b''.join(frames),
        sample_width=audio.get_sample_size(FORMAT),
        frame_rate=RATE,
        channels=CHANNELS
    )

    # Save the audio segment as an MP3 file
    recorded_audio.export(filename, format="mp3")
    recorded_audio.export()

def collect_eeg_data(filename):

    """Collect raw EEG data from the Muse2 stream and save to a CSV file."""

    global recording_active
    inlet = wait_for_eeg_source()  # Wait for EEG source before proceeding
    
    with open(filename, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        headers = ['Timestamp'] + [f'Ch{ch}' for ch in range(1, 5)] # Define and write your CSV headers based on our EEG data structure
        csv_writer.writerow(headers)

        while recording_active:
            """ Attempt to pull a chunk of EEG data from the LSL inlet. 
            `eeg_data` will contain the EEG measurements as a 2D list or array,
            where each sublist or row corresponds to a single sample and each column to a channel.
            `timestamps` will contain the corresponding timestamps for each sample.
            The function will wait for up to 1 second (`timeout=1`) to collect the data,
            and will gather a maximum of 12 samples (`max_samples=12`) during this period.
            If no data is available within the timeout period, `eeg_data` and `timestamps` will be empty."""
            eeg_data, timestamps = inlet.pull_chunk(timeout=1, max_samples=12) 
            if timestamps:
                for i, timestamp in enumerate(timestamps):
                    row = [timestamp] + eeg_data[i]
                    csv_writer.writerow(row)

def list_new_files(directory, extensions):
    new_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extensions):
                new_files.append(os.path.join(root, file))
    return new_files

def upload_new_files_and_move(folder_path, bucket_name, s3_folder, uploaded_subfolder):
    s3_client = boto3.client('s3')
    uploaded_folder_path = os.path.join(folder_path, uploaded_subfolder)

    # Check if the uploaded folder exists, create if not
    if not os.path.exists(uploaded_folder_path):
        os.makedirs(uploaded_folder_path)

    for file in os.listdir(folder_path):
        if file == uploaded_subfolder:
            continue  # Skip the uploaded directory itself

        full_path = os.path.join(folder_path, file)

        # Check if it's a file and not a directory
        if os.path.isfile(full_path):
            uploaded_file_path = os.path.join(uploaded_folder_path, file)

            # Check if the file has not been uploaded before
            if not os.path.exists(uploaded_file_path):
                with open(full_path, 'rb') as data:
                    s3_client.upload_fileobj(data, bucket_name, os.path.join(s3_folder, file))
                    print(f"File {file} uploaded successfully to {os.path.join(s3_folder, file)}")

                    # Move the file to the uploaded directory
                    shutil.move(full_path, uploaded_file_path)
                    print(f"File {file} moved to {uploaded_file_path}")


if __name__ == "__main__":

    recordings_dir = 'Recordings'

    if not os.path.exists(recordings_dir): # Create recordings directory if it doesn't exist
        os.makedirs(recordings_dir)
    
    # Setup file paths for EEG data and audio files, by attaching the current date and time to the filename
    eeg_filename = os.path.join(recordings_dir, f"eeg_data_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.csv")
    audio_filename = os.path.join(recordings_dir, f"audio_recording_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.mp3")

    print("Press spacebar to start/stop the recordings...")

    # Start the Muse LSL stream here before entering the waiting loop.
    start_muse_lsl_stream()

    listener = keyboard.Listener(on_press=toggle_recording) # The listener activate the function toggle_recording when spacebar is pressed
    listener.start()

    while not recording_active:
        # Wait for the first spacebar press to activate recording
        time.sleep(0.5)

    # Start threads for muse stream and audio recording
    eeg_thread = threading.Thread(target=collect_eeg_data, args=(eeg_filename,))
    audio_thread = threading.Thread(target=record_audio, args=(audio_filename,))
    eeg_thread.start()
    audio_thread.start()

    while recording_active:
        # Keep the main thread alive while recording is active
        time.sleep(0.5)
  
    audio_thread.join()
    listener.stop()

    print("Recording and streaming have finished.")

    # Usage
    upload_new_files_and_move("Recordings/", "dsr-jhana", "eeg/", "uploaded/")

Press spacebar to start/stop the recordings...
Searching for Muses, this may take up to 10 seconds...
Found device Muse-01EE, MAC Address 4D5E2EB4-EB72-1BE0-19D4-501ED06DF20A
Connecting to Muse-01EE: 4D5E2EB4-EB72-1BE0-19D4-501ED06DF20A...


2024-03-14 11:43:12.746 (  11.879s) [           44A3B]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-14 11:43:12.746 (  11.879s) [           44A3B]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-14 11:43:12.746 (  11.879s) [           44A3B]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-03-14 11:43:12.746 (  11.879s) [           44A3B]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-14 11:43:12.746 (  11.879s) [           44A3B]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-03-14 11:43:12.746 (  11.879s) [           44A3B]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-14 11:43:12.746 (  11.879s) [           44A3B]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-03-14 11:43:12.746 (  11.879s) [           44A3B]      netinterfaces.cpp:91    I

Connected.
Streaming EEG...
Starting recordings...
Searching for EEG source...
EEG source found.


2024-03-14 11:43:17.478 (  16.834s) [           44AD7]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-14 11:43:17.478 (  16.834s) [           44AD7]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-14 11:43:17.478 (  16.834s) [           44AD7]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-03-14 11:43:17.479 (  16.834s) [           44AD7]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-14 11:43:17.479 (  16.834s) [           44AD7]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-03-14 11:43:17.479 (  16.834s) [           44AD7]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-14 11:43:17.479 (  16.834s) [           44AD7]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-03-14 11:43:17.479 (  16.834s) [           44AD7]      netinterfaces.cpp:91    I

Stopping recordings...
Recording and streaming have finished.
File eeg_data_2024_03_14_10_28_32.csv uploaded successfully to eeg/eeg_data_2024_03_14_10_28_32.csv
File eeg_data_2024_03_14_10_28_32.csv moved to Recordings/uploaded/eeg_data_2024_03_14_10_28_32.csv
File audio_recording_2024_03_14_11_43_00.mp3 uploaded successfully to eeg/audio_recording_2024_03_14_11_43_00.mp3
File audio_recording_2024_03_14_11_43_00.mp3 moved to Recordings/uploaded/audio_recording_2024_03_14_11_43_00.mp3
File audio_recording_2024_03_14_11_06_05.mp3 uploaded successfully to eeg/audio_recording_2024_03_14_11_06_05.mp3
File audio_recording_2024_03_14_11_06_05.mp3 moved to Recordings/uploaded/audio_recording_2024_03_14_11_06_05.mp3
File eeg_data_2024_03_14_11_20_23.csv uploaded successfully to eeg/eeg_data_2024_03_14_11_20_23.csv
File eeg_data_2024_03_14_11_20_23.csv moved to Recordings/uploaded/eeg_data_2024_03_14_11_20_23.csv
File audio_recording_2024_03_14_10_33_33.mp3 uploaded successfully to eeg/audio_re

IsADirectoryError: [Errno 21] Is a directory: 'Recordings/uploaded'

Disconnected.
